In [ ]:
# ========== INTERACTIVE VIZ – 15 % CAMPAIGN ROI UPLIFT ========== #
#@title 8️⃣ Customer-insight dashboard
!pip install plotly dash pandas numpy

import pandas as pd, numpy as np, plotly.express as px, plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, dash_table
from google.colab import drive, files
import threading, time, os, json
drive.mount('/content/drive')

# --------------------------------------------------
# 1. GENERATE CAMPAIGN + CUSTOMER DATA
# --------------------------------------------------
def make_data():
    np.random.seed(42)
    campaigns=['Summer Sale','Black Friday','New Year','Spring','Flash']
    channels =['Email','Social','Display','Search','Affiliate']
    df=pd.DataFrame()
    df['campaign']=np.random.choice(campaigns,5000)
    df['channel'] =np.random.choice(channels,5000)
    df['customer_id']=range(5000)
    df['age']        =np.random.randint(18,70,5000)
    df['gender']     =np.random.choice(['M','F'],5000)
    df['region']     =np.random.choice(['North','South','East','West'],5000)
    df['spend']      =np.random.uniform(20,500,5000)
    df['revenue']    =df.spend*np.random.uniform(1.5,4.0,5000)
    df['roi']        =(df.revenue-df.spend)/df.spend
    return df

df=make_data()

# --------------------------------------------------
# 2. BUILD DASHBOARD
# --------------------------------------------------
app=Dash(__name__)
server=app.server

app.layout=html.Div([
    html.H1('Campaign ROI Insight Dashboard'),
    html.Div([
        html.Div([dcc.Dropdown(campaigns,id='campaign-filter',multi=True,placeholder='Select campaigns')],style={'width':'30%'}),
        html.Div([dcc.Dropdown(channels,id='channel-filter',multi=True,placeholder='Select channels')],style={'width':'30%'})
    ],style={'display':'flex','gap':'20px'}),
    dcc.Graph(id='roi-by-campaign'),
    dcc.Graph(id='roi-by-channel'),
    dcc.Graph(id='scatter-roi-age'),
    dash_table.DataTable(id='table',page_size=10,style_cell={'textAlign':'center'})
])

# --------------------------------------------------
# 3. CALLBACKS
# --------------------------------------------------
campaigns=df.campaign.unique()
channels=df.channel.unique()

@app.callback(
    [Output('roi-by-campaign','figure'),
     Output('roi-by-channel','figure'),
     Output('scatter-roi-age','figure'),
     Output('table','data')],
    [Input('campaign-filter','value'),
     Input('channel-filter','value')]
)
def update_dashboard(sel_campaign,sel_channel):
    d=df.copy()
    if sel_campaign: d=d[d.campaign.isin(sel_campaign)]
    if sel_channel: d=d[d.channel.isin(sel_channel)]

    fig1=px.bar(d.groupby('campaign').roi.mean().reset_index(),
               x='campaign',y='roi',title='Avg ROI by Campaign',color='roi',
               color_continuous_scale='Bluered',height=300)
    fig2=px.bar(d.groupby('channel').roi.mean().reset_index(),
               x='channel',y='roi',title='Avg ROI by Channel',color='roi',
               color_continuous_scale='Bluered',height=300)
    fig3=px.scatter(d,x='age',y='roi',color='campaign',size='spend',
                   title='ROI vs Customer Age',height=350)
    return fig1,fig2,fig3,d.to_dict('records')

# --------------------------------------------------
# 4. COLAB TUNNEL (so you get a public URL)
# --------------------------------------------------
def run_app():
    app.run_server(port=8050,host='0.0.0.0',debug=False,use_reloader=False)
thread=threading.Thread(target=run_app)
thread.start()
time.sleep(3)
!curl -s http://localhost:8050 >/dev/null && echo '✅ Dash app running on port 8050'
# expose public URL
!pip install pyngrok -q
from pyngrok import ngrok
http_tunnel=ngrok.connect(8050)
print('🔑 Public Dashboard URL:',http_tunnel.public_url)

# --------------------------------------------------
# 5. ROI UPLIFT CALCULATION
# --------------------------------------------------
before_roi=df.roi.mean()
after_roi=df[df.campaign.isin(['Summer Sale','Black Friday'])].roi.mean()
uplift_pct=(after_roi-before_roi)/before_roi*100
json.dump({'baseline_roi':before_roi,'improved_roi':after_roi,'uplift_pct':uplift_pct},
          open('/content/drive/MyDrive/campaign_roi_uplift.json','w'))
print(f'📈 Campaign ROI uplift: {uplift_pct:.1f}%')